In [ ]:
# basic libraries ...

import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# ultrlytics ...

!pip install ultralytics -q

In [ ]:
# yolo ...

from ultralytics import YOLO

In [ ]:
# delete all the files and directories recursively in the current working directory ...

!rm -rf *

# make directory ...

!mkdir /kaggle/working/datasets
!mkdir /kaggle/working/datasets/badodd
!mkdir /kaggle/working/datasets/badodd/labels
!mkdir /kaggle/working/datasets/badodd/labels/train
!mkdir /kaggle/working/datasets/badodd/images
!mkdir /kaggle/working/datasets/badodd/images/train
!mkdir /kaggle/working/datasets/badodd/images/test

In [ ]:
#  reference original files without duplicating their content ...

def all_files_in_folder_symlink(source_dir, target_dir):
    files = os.listdir(source_dir)
    
    for file in tqdm(files):
        source_file = os.path.join(source_dir, file)
        target_file = os.path.join(target_dir, file)
        os.symlink(source_file, target_file)

In [ ]:
# symbolic link function as above ...

all_files_in_folder_symlink("/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/labels/train","/kaggle/working/datasets/badodd/labels/train")
all_files_in_folder_symlink("/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/images/train","/kaggle/working/datasets/badodd/images/train")
all_files_in_folder_symlink("/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/images/test","/kaggle/working/datasets/badodd/images/test")

In [ ]:
# working directory structure ...

!tree -d

In [ ]:
# model part ...

In [ ]:
# configuration file that tells YOLO where to find the dataset and what objects to detect ...

file_content = """
path: /kaggle/working/datasets/badodd  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/train  # val images (relative to 'path')
test:  images/test

# Classes
names:
    0: auto_rickshaw
    1: bicycle
    2: bus
    3: car
    4: cart_vehicle
    5: construction_vehicle
    6: motorbike
    7: person
    8: priority_vehicle
    9: three_wheeler
    10: train
    11: truck
    12: wheelchair
"""

with open("yolov8.yaml", mode="w") as f:
    f.write(file_content)

In [ ]:
# Weights & Biases ...

import wandb
wandb.init(mode="disabled")
# wandb.init()

## Augmentations

In [ ]:
# Helper functions:
def load_annotation(annotation_file):
    annotations = []
    with open(annotation_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            # Parse the line to extract bounding box coordinates and class label
            data = line.strip().split()
            class_id = int(data[0])
            x_center = float(data[1])
            y_center = float(data[2])
            width = float(data[3])
            height = float(data[4])
            annotations.append([class_id, x_center, y_center, width, height])
    return annotations

def save_annotation(annotation_file, annotations):
    with open(annotation_file, 'w') as file:
        for box in annotations:
            # Convert bounding box coordinates to string format and write to file
            line = ' '.join([str(coord) for coord in box]) + '\n'
            file.write(line)



In [ ]:
import cv2
import numpy as np

# Load original image and annotation
original_image_path = '/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/images/train/sylhet4_9794.jpg'
original_annotation_path = original_image_path.replace('images', 'labels').replace('.jpg', '.txt')
original_image = cv2.imread(original_image_path)
original_annotation = load_annotation(original_annotation_path)
print(original_annotation)

# Apply image augmentation (e.g., rotation)
angle = 15
rows, cols = original_image.shape[:2]
rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
augmented_image = cv2.warpAffine(original_image, rotation_matrix, (cols, rows))

# Update bounding box coordinates in the annotation
# Example: update bounding box coordinates for rotation
# (Assuming the bounding box format is [x_center, y_center, width, height])
updated_annotation = []
for box in original_annotation:
    class_id, x_center, y_center, width, height = box
    
    # Rotate bounding box coordinates
    x_min = x_center - width / 2
    y_min = y_center - height / 2
    x_max = x_center + width / 2
    y_max = y_center + height / 2

    # Rotate bounding box coordinates
    new_x_min = x_min * np.cos(np.radians(angle)) - y_min * np.sin(np.radians(angle))
    new_y_min = x_min * np.sin(np.radians(angle)) + y_min * np.cos(np.radians(angle))
    new_x_max = x_max * np.cos(np.radians(angle)) - y_max * np.sin(np.radians(angle))
    new_y_max = x_max * np.sin(np.radians(angle)) + y_max * np.cos(np.radians(angle))

    # Recalculate width and height
    new_width = new_x_max - new_x_min
    new_height = new_y_max - new_y_min

    # Recalculate center coordinates
    new_x_center = new_x_min + new_width / 2
    new_y_center = new_y_min + new_height / 2

    updated_annotation.append([class_id, new_x_center, new_y_center, new_width, new_height])

    
    
    
    # Rotate bounding box center coordinates
#     new_x_center = x_center * np.cos(np.radians(angle)) - x_center * np.sin(np.radians(angle))
#     new_y_center = y_center * np.sin(np.radians(angle)) + y_center * np.cos(np.radians(angle))
#     updated_annotation.append([class_id, new_x_center, new_y_center, width, height])

# Save augmented image and updated annotation
cv2.imwrite('augmented_image.jpg', augmented_image)
save_annotation('updated_annotation.txt', updated_annotation)



In [ ]:
import matplotlib.pyplot as plt

# Load augmented image
augmented_image = cv2.imread('/kaggle/working/augmented_image.jpg')

# Read the annotations from the text file
annotations_file_path = '/kaggle/working/updated_annotation.txt'
with open(annotations_file_path, 'r') as f:
    annotations_data = f.readlines()

# Parse annotations and draw bounding boxes
for line in annotations_data:
    class_id, x_center, y_center, width, height = map(float, line.strip().split())
    
    # Convert YOLO format to bounding box coordinates
    img_height, img_width = original_image.shape[:2]
    x = (x_center - width / 2) * img_width
    y = (y_center - height / 2) * img_height
    w = width * img_width
    h = height * img_height
    
    # Draw bounding box on the image
    original_image = cv2.rectangle(augmented_image, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)

# Convert BGR to RGB for Matplotlib
original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# Display the image with bounding boxes using Matplotlib
plt.figure()
plt.imshow(original_image_rgb)
plt.axis('off')
plt.title('Image with Bounding Boxes')
plt.show()



In [ ]:
import cv2
import matplotlib.pyplot as plt

# Read the original image
original_image_path = '/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/images/train/sylhet4_9794.jpg'
original_image = cv2.imread(original_image_path)

# Read the annotations from the text file
annotations_file_path = original_image_path.replace('images','labels').replace('.jpg','.txt')
with open(annotations_file_path, 'r') as f:
    annotations_data = f.readlines()

# Parse annotations and draw bounding boxes
for line in annotations_data:
    class_id, x_center, y_center, width, height = map(float, line.strip().split())
    
    # Convert YOLO format to bounding box coordinates
    img_height, img_width = original_image.shape[:2]
    x = (x_center - width / 2) * img_width
    y = (y_center - height / 2) * img_height
    w = width * img_width
    h = height * img_height
    
    # Draw bounding box on the image
    original_image = cv2.rectangle(original_image, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)

# Convert BGR to RGB for Matplotlib
original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# Display the image with bounding boxes using Matplotlib
plt.imshow(original_image_rgb)
plt.axis('off')
plt.title('Image with Bounding Boxes')
plt.show()


In [ ]:
# yolov8 small model, configuration file and run the model in 3 epochs ... 

# Adam, AdamW, NAdam, RAdam, RMSProp, SGD, auto
model_RMS = YOLO("yolov8n.pt")
model_RMS.train(
    data="/kaggle/working/yolov8.yaml", 
    
    # Augmentation
    perspective=0.3,
    mosaic=0.8,
    scale=0.3,
    translate=0.4,
    
    # Training Parameters
    epochs=100,
    batch=16,
#     workers=2,
    val=True,
    
    # Optimization parameters
    lr0=0.0001,
#     patience=3,
    optimizer="RMSProp",
)

In [ ]:
# get prediction boxes string according to the submission format ...

def get_prediction_string(boxes, scores, classes):
    pred_strs = []
    for i, score in enumerate(scores):
        single_pred_str = ""
        single_pred_str += str(float(classes[i])) + " " + str(float(score)) + " "
        
        x_center , y_center, width,height = boxes[i]
        x1 = float(x_center) - (float(width) / 2)
        y1 = float(y_center) - (float(height) / 2)
        width = float(width)
        height = float(height)
        #single_pred_str += " ".join(str(float(x)) for x in boxes[i])
        single_pred_str +=  f"{x1} {y1} {width} {height}"
        
        pred_strs.append(single_pred_str)
    ans = ','.join(map(str, pred_strs))
    if len(ans):
        return ans
#     the solution metrics faield in case of a NaN, '' (empty). So, return "0 0 0 0 0 0" for NaN, '' (empty) string
    return "0 0 0 0 0 0"

# get the predcition in id, ImageID, PredictionString_pred foramt ...

def get_prediction_entry(i, filename, boxes, scores, classes):
    return {
        "id": i, # strating from 0 ...
        "ImageID": filename.split('.')[0], # before the extension ...
        "PredictionString_pred": get_prediction_string(boxes, scores, classes)
    }

# Directory path ...
test_directory = "/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/images/test"

# Load the model ...
model = YOLO('/kaggle/working/runs/detect/train3/weights/best.pt')

# do the inference ...

def predict_all_files(test_directory):
    predictions = []
    for i,filename in tqdm(enumerate(os.listdir(test_directory))):
        if filename.endswith(".jpg"):
            filepath = os.path.join(test_directory, filename)
            results = model.predict(source=filepath, conf=0.50, verbose=False)
            boxes = results[0].boxes.xywhn
            scores = results[0].boxes.conf
            classes = results[0].boxes.cls
            prediction = get_prediction_entry(i, filename, boxes, scores, classes)
            predictions.append(prediction)
#             to csv format ...
    predictions_df = pd.DataFrame(predictions)
    predictions_df.to_csv("submission.csv", index=False)

# call the inference function ...
predict_all_files(test_directory)

In [ ]:
# load the submission dataframe ....

submission_df = pd.read_csv('/kaggle/working/submission.csv')
submission_df.tail()

In [ ]:
submission_df.shape()